# Orders

✏️ **Exercise**

Today, we will investigate the **orders**, and their associated **review score**.

👉 Our goal is to create a DataFrame with the following features:


| feature_name 	| type 	| description 	|
|:---	|:---:	|:---	|
| `order_id` 	| str 	| the id of the order 	|
| `wait_time` 	| float 	| the number of days between order_date and delivered_date 	|
| `expected_wait_time` 	| float 	| the number of days between order_date and estimated_delivery_date 	|
| `delay_vs_expected` 	| float 	| if the actual delivery date is later than the estimated delivery date, returns the number of days between the two dates, otherwise return 0 	|
| `order_status` 	| str 	| the status of the order 	|
| `dim_is_five_star` 	| int 	| 1 if the order received a five-star review, 0 otherwise 	|
| `dim_is_one_star` 	| int 	| 1 if the order received a one_star, 0 otherwise 	|
| `review_score` 	| int 	| from 1 to 5 	|
| `number_of_products` 	| int 	| number of products that the order contains 	|
| `number_of_sellers` 	| int 	| number of sellers involved in the order 	|
| `price` 	| float 	| total price of the order paid by customer 	|
| `freight_value` 	| float 	| value of the freight paid by customer 	|
| `distance_customer_seller` 	| float 	| the distance in km between customer and seller (optional) 	|  
  
⚠️ We also want to filter out "non-delivered" orders, unless explicitly specified, otherwise we cannot compute the potential delays.

❓ **Your challenge**: 

- Implement each feature as a separate method within the `Order` class available at `olist/order.py`
- Then, create a method `get_training_data()` that returns the complete DataFrame.

💡 Suggested methodology:
- Use the notebook below to write and test your code step-by-step first
- Then copy the code into `order.py` once you are certain of your code logic
- Focus on the data manipulation logic now, we will analyse the dataset visually in the next challenges

🔥 Notebook best practices (must-read) 👇

<details>
    <summary>▸ <i>click here</i></summary>

From now on, exploratory notebooks are going to become pretty long, and we strongly advise you to follow these notebook principles:
- Code your logic so that your Notebook can always be ran from top to bottom without crashing (Cell --> Run All)
- Name your variables carefully 
- Use dummy names such as `tmp` or `_` for intermediary steps when you know you won't need them for long
- Clear your code and merge cells when relevant to minimize Notebook size (`Shift-M`)
- Hide your cell output if you don't need to see it anymore (double-click on the red `Out[]:` section to the left of your cell).
- Make heavy use of jupyter nbextention `Collapsable Headings` and `Table of Content` (call a TA if you can't find them)
- Use the following shortcuts 
    - `a` to insert a cell above
    - `b` to insert a cell below
    - `dd` to delete a cell
    - `esc` and `arrows` to move between cells
    - `Shift-Enter` to execute cell and move focus to the next one
    - use `Shift + Tab` when you are between method brackets e.g. `groupby()` to get the docs! Repeat a few times to open it permanently

</details>





In [1]:
# Auto reload imported module everytime a jupyter cell is executed (handy for olist.order.py updates)
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Import olist data
from olist.data import Olist
olist = Olist()
data = olist.get_data()

In [4]:
# What datasets do we have access to now ? 
data.keys()

dict_keys(['sellers', 'order_reviews', 'order_items', 'customers', 'orders', 'order_payments', 'product_category_name_translation', 'products', 'geolocation'])

In [5]:
orders = data['orders'].copy() # good practice to be sure not to modify your `data` variable

assert(orders.shape == (99441, 8))

## 1. Code `order.py`

### a) `get_wait_time`
    ❓ Return a Dataframe with:
           order_id, wait_time, expected_wait_time, delay_vs_expected, order_status


🎁 We give you the pseudo-code below 👇 for this first method:

> 1. Inspect the `orders` dataframe
2. Filter the dataframe on `delivered orders`
3. Handle `datetime`
    - Take time to understand what python [`datetime`](https://docs.python.org/3/library/datetime.html) objects are
    - and convert dates from "string" type to "pandas.datetime' using [`pandas.to_datetime()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html)
4. Compute `wait_time`
5. Compute `expected_wait_time`
6. Compute `delay_vs_expected`
7. Check the new dataframe 
8. Once you are satisfied with your code, you can carefully copy-paste it from the notebook to to `olist/order.py`

In [6]:
orders = orders[orders["order_status"] == "delivered"]
orders["order_purchase_timestamp"] = pd.to_datetime(orders["order_purchase_timestamp"])
orders["order_approved_at"] = pd.to_datetime(orders["order_approved_at"])
orders["order_delivered_carrier_date"] = pd.to_datetime(orders["order_delivered_carrier_date"])
orders["order_delivered_customer_date"] = pd.to_datetime(orders["order_delivered_customer_date"])
orders["order_estimated_delivery_date"] = pd.to_datetime(orders["order_estimated_delivery_date"])
orders["wait_time"] = orders["order_delivered_customer_date"] - orders["order_purchase_timestamp"]
orders["expected_wait_time"] = orders["order_estimated_delivery_date"] - orders["order_purchase_timestamp"]
orders["delay_vs_expected"] = orders["wait_time"] - orders["expected_wait_time"]
orders = orders[["order_id", "wait_time", "expected_wait_time", "delay_vs_expected", "order_status"]]
orders

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8 days 10:28:40,15 days 13:03:27,-8 days +21:25:13,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13 days 18:46:08,19 days 03:18:23,-6 days +15:27:45,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9 days 09:27:40,26 days 15:21:11,-18 days +18:06:29,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13 days 05:00:36,26 days 04:31:54,-13 days +00:28:42,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2 days 20:58:23,12 days 02:41:21,-10 days +18:17:02,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8 days 05:13:56,18 days 14:05:55,-11 days +15:08:01,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22 days 04:38:58,23 days 11:01:02,-2 days +17:37:56,delivered
99438,83c1379a015df1e13d02aae0204711ab,24 days 20:37:34,30 days 09:13:17,-6 days +11:24:17,delivered
99439,11c177c8e97725db2631073c19f07b62,17 days 02:04:27,37 days 02:31:33,-21 days +23:32:54,delivered


👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [7]:
orders["wait_time"] = orders["wait_time"].dt.days
orders["expected_wait_time"] = orders["expected_wait_time"].dt.days
orders["delay_vs_expected"] = orders["delay_vs_expected"].dt.days
convert_time = lambda x: 0 if x <= 0 else x
orders["delay_vs_expected"]
orders

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15,-8.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19,-6.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26,-18.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.0,26,-13.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.0,12,-10.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.0,18,-11.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.0,23,-2.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.0,30,-6.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.0,37,-21.0,delivered


In [8]:
orders["expected_wait_time"] = orders["expected_wait_time"].astype("float64")

In [9]:
orders

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15.0,-8.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19.0,-6.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26.0,-18.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.0,26.0,-13.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.0,12.0,-10.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.0,18.0,-11.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.0,23.0,-2.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.0,30.0,-6.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.0,37.0,-21.0,delivered


In [10]:
# Test your code here
from olist.order import Order
Order().get_wait_time()

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15.0,-8.0,delivered
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19.0,-6.0,delivered
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26.0,-18.0,delivered
3,949d5b44dbf5de918fe9c16f97b45f8a,13.0,26.0,-13.0,delivered
4,ad21c59c0840e6cb83a9ceb5573f8159,2.0,12.0,-10.0,delivered
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,8.0,18.0,-11.0,delivered
99437,63943bddc261676b46f01ca7ac2f7bd8,22.0,23.0,-2.0,delivered
99438,83c1379a015df1e13d02aae0204711ab,24.0,30.0,-6.0,delivered
99439,11c177c8e97725db2631073c19f07b62,17.0,37.0,-21.0,delivered


In [11]:
from nbresult import ChallengeResult
result = ChallengeResult('wait_time', shape=Order().get_wait_time().shape)
result.write(); print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/chongxe1991/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/chongxe1991/code/chongxe1991/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_wait_time.py::TestWaitTime::test_wait_time PASSED             [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/wait_time.pickle

git commit -m 'Completed wait_time step'

git push origin master


### b) `get_review_score`
     ❓ Returns a DataFrame with:
        order_id, dim_is_five_star, dim_is_one_star, review_score

dim_is_$N$_star should contain `1` if review_score=$N$ and `0` otherwise 

<details>
    <summary markdown='span'>Hints</summary>

Think about `Series.map()` or `DataFrame.apply()`
    
</details>

👉 We load the `reviews` for you

In [12]:
reviews = data['order_reviews'].copy()
assert(reviews.shape == (99224,7))
reviews

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
...,...,...,...,...,...,...,...
99219,574ed12dd733e5fa530cfd4bbf39d7c9,2a8c23fee101d4d5662fa670396eb8da,5,NaN,NaN,2018-07-07 00:00:00,2018-07-14 17:18:30
99220,f3897127253a9592a73be9bdfdf4ed7a,22ec9f0669f784db00fa86d035cf8602,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42
99221,b3de70c89b1510c4cd3d0649fd302472,55d4004744368f5571d1f590031933e4,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43
99222,1adeb9d84d72fe4e337617733eb85149,7725825d039fc1f0ceb7635e3f7d9206,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13


In [13]:
assign_5 = lambda x: 1 if (x == 5) else 0
assign_1 = lambda x: 1 if (x == 1) else 0
reviews["dim_is_five_star"] = reviews["review_score"].apply(assign_5)
reviews["dim_is_one_star"] = reviews["review_score"].apply(assign_1)
reviews = reviews[["order_id", "dim_is_five_star", "dim_is_one_star", "review_score"]]
reviews

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


Once again, 

👀 Check the dataframe you've just created. <br/> 

💪 When your code works, commit it to `olist/order.py` <br/>

🧪 Now, test it by running the following cell 👇 

In [14]:
# Test your code here
from olist.order import Order
Order().get_review_score()

,order_id,dim_is_five_star,dim_is_one_star,review_score
0,73fc7af87114b39712e6da79b0a377eb,0,0,4
1,a548910a1c6147796b98fdf73dbeba33,1,0,5
2,f9e4b658b201a9f2ecdecbb34bed034b,1,0,5
3,658677c97b385a9be170737859d3511b,1,0,5
4,8e6bfb81e283fa7e4f11123a3fb894f1,1,0,5
...,...,...,...,...
99219,2a8c23fee101d4d5662fa670396eb8da,1,0,5
99220,22ec9f0669f784db00fa86d035cf8602,1,0,5
99221,55d4004744368f5571d1f590031933e4,1,0,5
99222,7725825d039fc1f0ceb7635e3f7d9206,0,0,4


In [15]:
from nbresult import ChallengeResult
result = ChallengeResult('review_score', shape=Order().get_review_score().shape)
result.write(); print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/chongxe1991/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/chongxe1991/code/chongxe1991/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_review_score.py::TestReviewScore::test_review_score PASSED    [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/review_score.pickle

git commit -m 'Completed review_score step'

git push origin master


### c) `get_number_products`:
     ❓ Returns a DataFrame with:
        order_id, number_of_products (total number of products per order)

In [16]:
order_items = data["order_items"].copy()

In [17]:
order_items2 = order_items.groupby("order_id").count()
order_items2.rename(columns = {"order_item_id": "number_of_products"}, inplace = True)
order_items2.reset_index(inplace = True)

In [18]:
number_products = order_items2[["order_id", "number_of_products"]]
number_products

,order_id,number_of_products
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


🧪 Same routine: 
* check your dataframe, 
* commit your code to `olist/order.py`
* and check that it truly works.

In [19]:
from nbresult import ChallengeResult
result = ChallengeResult('number_products', shape=Order().get_number_products().shape)
result.write(); print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/chongxe1991/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/chongxe1991/code/chongxe1991/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_number_products.py::TestNumberProducts::test_review_score PASSED [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/number_products.pickle

git commit -m 'Completed number_products step'

git push origin master


### d) `get_number_sellers`:
     ❓ Returns a DataFrame with:
        order_id, number_of_sellers (total number of unique sellers per order)
        
<details>
    <summary>▸ <i>Hint</i></summary>

`pd.Series.nunique()`
</details>

In [20]:
order_items = data["order_items"].copy()
order_items = order_items.groupby("order_id").nunique()
order_items.reset_index(inplace = True)
order_items.rename(columns = {"seller_id": "number_of_sellers"}, inplace = True)
number_sellers = order_items[["order_id", "number_of_sellers"]]
number_sellers

,order_id,number_of_sellers
0,00010242fe8c5a6d1ba2dd792cb16214,1
1,00018f77f2f0320c557190d7a144bdd3,1
2,000229ec398224ef6ca0657da4fc703e,1
3,00024acbcdf0a6daa1e931b038114c75,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1
...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1
98662,fffcd46ef2263f404302a634eb57f7eb,1
98663,fffce4705a9662cd70adb13d4a31832d,1
98664,fffe18544ffabc95dfada21779c9644f,1


In [21]:
from nbresult import ChallengeResult
result = ChallengeResult('number_sellers', shape=Order().get_number_sellers().shape)
result.write(); print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/chongxe1991/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/chongxe1991/code/chongxe1991/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_number_sellers.py::TestNumberSellers::test_number_seller PASSED [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/number_sellers.pickle

git commit -m 'Completed number_sellers step'

git push origin master


### e) `get_price_and_freight`
     Returns a DataFrame with:
        order_id, price, freight_value

<details>
    <summary>▸ <i>Hint</i></summary>

`pd.Series.agg()` allows you to apply one transformation method per column of your groupby object
</details>

In [22]:
order_items = data["order_items"].copy()
price_freight = order_items.groupby("order_id").sum()
price_freight.reset_index(inplace = True)
price_freight = price_freight[["order_id", "price", "freight_value"]]
price_freight

,order_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14
...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,299.99,43.41
98662,fffcd46ef2263f404302a634eb57f7eb,350.00,36.53
98663,fffce4705a9662cd70adb13d4a31832d,99.90,16.95
98664,fffe18544ffabc95dfada21779c9644f,55.99,8.72


In [23]:
from nbresult import ChallengeResult
result = ChallengeResult('price', shape=Order().get_price_and_freight().shape)
result.write(); print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/chongxe1991/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/chongxe1991/code/chongxe1991/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_price.py::TestPrice::test_price PASSED                        [100%]

============================== 1 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/price.pickle

git commit -m 'Completed price step'

git push origin master


### e) [OPTIONAL] `get_distance_seller_customer` 
**(Try  to code this function only after finishing today's challenges - Skip to next section)**

    ❓ Returns a Dataframe with:
        order_id, distance_seller_customer (the distance in km between customer and seller)

💡Have a look at the `haversine_distance` formula we coded for you in the `olist.utils` module

👀 Check your new dataframe and commit your code to olist/order.py when it works. 

In [24]:
# YOUR CODE HERE

🧪  Test your code

In [25]:
from nbresult import ChallengeResult

result = ChallengeResult('distance',
    mean = Order().get_distance_seller_customer()['distance_seller_customer'].mean())
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/chongxe1991/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/chongxe1991/code/chongxe1991/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: anyio-3.4.0
collecting ... collected 1 item

tests/test_distance.py::TestDistance::test_distance PASSED               [100%]

============================== 1 passed in 0.10s ===============================


💯 You can commit your code:

git add tests/distance.pickle

git commit -m 'Completed distance step'

git push origin master


## 2. All at once: `get_training_data`

❓ Time to code `get_training_data` making use of your previous coded methods, to gather all order features in one table

In [26]:
# orders, reviews, number_products, number_sellers, price_freight
all_df = orders.merge(reviews, on = "order_id").merge(number_products, on = "order_id").merge(number_sellers, on = "order_id").merge(price_freight, on = "order_id")
all_df

,order_id,wait_time,expected_wait_time,delay_vs_expected,order_status,dim_is_five_star,dim_is_one_star,review_score,number_of_products,number_of_sellers,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,8.0,15.0,-8.0,delivered,0,0,4,1,1,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,13.0,19.0,-6.0,delivered,0,0,4,1,1,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,9.0,26.0,-18.0,delivered,1,0,5,1,1,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,13.0,26.0,-13.0,delivered,1,0,5,1,1,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,2.0,12.0,-10.0,delivered,1,0,5,1,1,19.90,8.72
...,...,...,...,...,...,...,...,...,...,...,...,...
96356,9c5dedf39a927c1b2549525ed64a053c,8.0,18.0,-11.0,delivered,1,0,5,1,1,72.00,13.08
96357,63943bddc261676b46f01ca7ac2f7bd8,22.0,23.0,-2.0,delivered,0,0,4,1,1,174.90,20.10
96358,83c1379a015df1e13d02aae0204711ab,24.0,30.0,-6.0,delivered,1,0,5,1,1,205.99,65.02
96359,11c177c8e97725db2631073c19f07b62,17.0,37.0,-21.0,delivered,0,0,2,2,1,359.98,81.18


In [27]:
all_df.dropna(inplace = True)

🧪  Test it below

In [28]:
from nbresult import ChallengeResult
from olist.order import Order
data = Order().get_training_data()

result = ChallengeResult('training',
    shape=data.shape,
    columns=sorted(list(data.columns))
)
result.write()
print(result.check())

============================= test session starts ==============================
platform linux -- Python 3.8.12, pytest-6.2.5, py-1.11.0, pluggy-1.0.0 -- /home/chongxe1991/.pyenv/versions/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /home/chongxe1991/code/chongxe1991/data-challenges/04-Decision-Science/02-Statistical-Inference/01-Orders
plugins: anyio-3.4.0
collecting ... collected 2 items

tests/test_training.py::TestTraining::test_training_data_columns PASSED  [ 50%]
tests/test_training.py::TestTraining::test_training_data_shape PASSED    [100%]

============================== 2 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/training.pickle

git commit -m 'Completed training step'

git push origin master


🏁 Congratulations! 

💾 Commit and push your notebook before starting the next challenge.